In [7]:
# Reference :
# https://github.com/richliao/textClassifier/blob/master/textClassifierHATT.py

import numpy as np
import pandas as pd
from collections import defaultdict
import re

import sys
import os

os.environ['KERAS_BACKEND']='theano'

In [77]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
# from keras import initializations
from keras import initializers

In [46]:
MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [23]:
import pickle
def save_pickle(path, X):
    with open(path, 'wb') as f:
        pickle.dump(X, f)
def open_pickle(path):
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train = open_pickle("../../data/imdb/imdb_original_preprocessed_xtrain.pickle")
X_test = open_pickle("../../data/imdb/imdb_original_preprocessed_xtest.pickle")
y_train = open_pickle("../../data/imdb/imdb_original_preprocessed_ytrain.pickle")
y_test = open_pickle("../../data/imdb/imdb_original_preprocessed_ytest.pickle")

In [29]:
from nltk import tokenize
from textblob import TextBlob

In [42]:
reviews = [] #sentences

for i in range(len(X_train)):
    sentences = TextBlob(X_train[i]).raw_sentences
    reviews.append(sentences)

In [43]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_train)

C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\keras\preprocessing\text.py:157: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [47]:
data = np.zeros((len(X_train), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [48]:
for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j<MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
                    data[i,j,k] = tokenizer.word_index[word]
                    k=k+1

In [53]:
word_index = tokenizer.word_index

In [59]:
# labels = to_categorical(np.asarray(y_train))
labels = np.asarray(y_train)

In [50]:
print('Total %s unique tokens.' % len(word_index))

Total 85439 unique tokens.


In [52]:
print('Shape of data tensor:', data.shape)

Shape of data tensor: (25000, 15, 100)


In [60]:
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (25000,)


In [61]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [62]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [64]:
print('Number of positive and negative reviews in training and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in training and validation set
10026
2474


In [69]:
GLOVE_DIR = "../../data/glove.6B"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), 'rb')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [72]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [80]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SENT_LENGTH,
                           trainable=True)

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences) 
sentEncoder = Model(sentence_input, l_lstm)

C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\keras\layers\recurrent.py:2024: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


In [81]:
review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)

C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\keras\layers\recurrent.py:2024: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


In [90]:
preds = Dense(1, activation='sigmoid')(l_lstm_sent)
model = Model(review_input, preds)

In [91]:
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['acc'])

In [92]:
print("model fitting - Hierarchical LSTM")
print(model.summary())

model fitting - Hierarchical LSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 15, 100)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 200)           8704800   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 8,945,801
Trainable params: 8,945,801
Non-trainable params: 0
_________________________________________________________________
None


In [95]:
model.fit(x_train,y_train, validation_data=(x_val,y_val),
         epochs=10, batch_size=50, verbose=1)

INFO (theano.gof.compilelock): Refreshing lock C:\Users\Anneke Hidayat\AppData\Local\Theano\compiledir_Windows-10-10.0.17134-SP0-Intel64_Family_6_Model_78_Stepping_3_GenuineIntel-3.6.4-64\lock_dir\lock


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
  500/20000 [..............................] - ETA: 42:48 - loss: 0.8178 - acc: 0.4640

KeyboardInterrupt: 